In [21]:
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn, optim
import pickle
import os
from mnist import MNIST
from torch.utils.data import DataLoader, TensorDataset


device(type='cuda')

In [22]:
# 定义数据转换
transform = transforms.Normalize((0.5,), (0.5,))

# 加载本地数据
mndata = MNIST('./data')
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

In [23]:
# 将数据转换为张量
train_images = torch.tensor(train_images).view(-1, 1, 28, 28).float() / 255.0
train_labels = torch.tensor(train_labels).long()
test_images = torch.tensor(test_images).view(-1, 1, 28, 28).float() / 255.0
test_labels = torch.tensor(test_labels).long()

# 应用数据转换
# train_images = transform(train_images)
# test_images = transform(test_images)

# 创建数据集
train_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

# 划分训练集为训练集与验证集
train_len = int(len(train_dataset) * 0.8)
val_len = len(train_dataset) - train_len
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=6)

In [15]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化LeNet网络
lenet = LeNet()

In [16]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# 实例化AlexNet网络
alexnet = AlexNet()

In [28]:
import torch.nn.functional as F

# 选择网络 (LeNet 或 AlexNet)
model = lenet  # 或者 model = alexnet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # 梯度置零
        optimizer.zero_grad()

        # 正向传播、反向传播、优化
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 打印统计信息
        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000个小批次打印一次
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [25]:
def evaluate(model, data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")

print("Validation set:")
evaluate(model, val_loader)
print("Test set:")
evaluate(model, test_loader)

Validation set:
Accuracy: 98.82%
Test set:
Accuracy: 98.90%


In [27]:
print(enumerate(train_loader, 0))